<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/HI_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Functions

In [ ]:
# Preprocess data function
def preprocess_data(matched_data):
    matched_data['distance'] = 0
    matched_data = matched_data[[str(i) == '20' for i in matched_data["depth_adj_bottom"]] ]
    matched_data['imp_c_float'] = matched_data['imp_c'].astype(float)

    # Select ID fields and numeric columns
    id_fields = matched_data[['source_dataset', 'island', 'dist_id', 'soil_column_id', 'unique_id', 'depth_top',
                              'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude', 'x_sample', 'y_sample',
                              'x_driver', 'y_driver']]
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']
    numeric_cols = matched_data[keep_cols].replace('', np.nan).astype(float).fillna(0)

    # Scale numeric columns
    scaler = MinMaxScaler()
    scaled_numeric_cols = scaler.fit_transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)

    # Scale imp_c
    min_c = matched_data['imp_c_float'].min()
    max_c = matched_data['imp_c_float'].max()
    scaled_imp_c = (matched_data['imp_c_float'] - min_c) / (max_c - min_c)

    # Combine ID fields with scaled numeric data
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)
    numeric_df['imp_c_scaled'] = scaled_imp_c

    return numeric_df, scaler, min_c, max_c

In [ ]:
# Train model function
def train_model(preprocessed_data):
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']
    X = preprocessed_data[keep_cols]
    y = preprocessed_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build and compile model
    model = Sequential([
        Dense(256, activation='relu', input_dim=X_train.shape[1]),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Train model
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

In [ ]:
# Predict on reserve function
def predict_on_reserve(preprocessed_data, model, min_c, max_c):
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']
    X = preprocessed_data[keep_cols]
    predictions = model.predict(X)

    inversed_predictions = predictions * (max_c - min_c) + min_c
    inversed_truth = preprocessed_data['imp_c_scaled'] * (max_c - min_c) + min_c

    df_out = preprocessed_data.copy()
    df_out['predictions'] = predictions
    df_out['inversed_predictions'] = inversed_predictions
    df_out['inversed_imp_c'] = inversed_truth

    return df_out

In [ ]:
# Evaluate model function
def evaluate_model(model, X_test, y_test, inversed_predictions, inversed_truth):
    test_loss = model.evaluate(X_test, y_test)
    r_squared = r2_score(inversed_truth, inversed_predictions)

    print("Test Loss:", test_loss)
    print("R-Squared Score:", r_squared)

    mae = mean_absolute_error(inversed_truth, inversed_predictions)
    rmse = mean_squared_error(inversed_truth, inversed_predictions, squared=False)

    print("Mean Absolute Error (MAE):", mae)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Scatter plot of true vs predicted values
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=inversed_truth, y=inversed_predictions)
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("True vs Predicted Values")
    plt.plot([min(inversed_truth), max(inversed_truth)], [min(inversed_truth), max(inversed_truth)], 'r')
    plt.show()

    # Residual plot
    residuals = inversed_truth - inversed_predictions
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel("Residuals")
    plt.title("Distribution of Residuals")
    plt.show()


In [ ]:
# Cross-validation and model training
def build_model(input_shape, output_shape):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_shape),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(output_shape, activation='linear')
    ])
    return model

In [ ]:
# Define function for rescaling
def rescale_to_minus_one_one(array):
    return 2 * (array - array.min()) / (array.max() - array.min()) - 1


In [ ]:
def train_model_a(preprocess_data):

    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    X = preprocess_data[keep_cols]

    y = preprocess_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define model
    def build_model(input_shape):
        model = Sequential([
            Dense(256, activation='relu', input_dim=input_shape),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(512, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1, activation='linear')
        ])
        return model

    model = build_model(X_train.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

# Load and preprocess data

In [ ]:
# Load data
gdf = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/annotated_combo_imputed_SOC.gpkg')
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')
soils_csv = pd.read_csv('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')
labeled_dist = pd.read_csv('/content/drive/MyDrive/hawaii_soils/labeled_distr_annote.csv')

In [ ]:
# Preprocess soils data
soils_csv = soils_csv.dropna(subset=['latitude', 'longitude'])
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")

# Merge and transform GeoDataFrames
soils_gpd = pd.merge(soils_gpd, gdf[['dist_id', 'unique_id']], on='unique_id', how='inner')
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

In [ ]:
# Additional preprocessing steps
drivers_gpd['x_driver'] = drivers_gpd.geometry.x
drivers_gpd['y_driver'] = drivers_gpd.geometry.y
soils_gpd['x_sample'] = soils_gpd.geometry.x
soils_gpd['y_sample'] = soils_gpd.geometry.y

In [47]:
soils_gpd = pd.merge(labeled_dist, soils_gpd, on='dist_id')

In [ ]:
# Reproject both GeoDataFrames to the same CRS
soils_buffered = soils_gpd.to_crs(epsg=32604)
drivers_gpd = drivers_gpd.to_crs(soils_buffered.crs)

# Apply buffer in the same CRS
soils_buffered['x_sample'] = soils_buffered.geometry.x
soils_buffered['y_sample'] = soils_buffered.geometry.y
soils_buffered.geometry = soils_buffered.geometry.buffer(1000)

# Perform spatial join
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', predicate='intersects')

In [41]:
matches_within_distance = pd.merge(labeled_dist, matches_within_distance, on='dist_id')

In [38]:
# Preprocess data and get scaled values
numeric_df, scaler, min_c, max_c = preprocess_data(matches_within_distance)


/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [54]:
soils_gpd.columns

Index(['dist_id', 'labeled_dist_h', 'labeled_dist_a', 'source_dataset',
       'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom',
       'depth_adj_bottom', 'latitude', 'longitude', 'c_perc', 'imp_c',
       'bulk_density', 'imp_bd', 'hole_id', 'geometry', 'x_sample',
       'y_sample'],
      dtype='object')

In [98]:
import pandas as pd
import numpy as np

# Assuming numeric_df needs to have dist_id included, merge or join it if necessary
# numeric_df = numeric_df.merge(matches_within_distance[['dist_id']], on='some_common_column', how='left')

dict_list = []

for dist_id in matches_within_distance['dist_id'].unique():
    dist_df = pd.DataFrame({
        "dist_id": [dist_id],
        "labeled_dist_h": soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['labeled_dist_h'].unique(),
        "labeled_dist_a": soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['labeled_dist_a'].unique(),
        "imp_c": [soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['imp_c'].tolist()],
        "imp_c_quantile": [np.quantile(soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['imp_c'], np.linspace(0, 1, 10)).tolist()]
    })
    dict_list.append(dist_df)

# Concatenate all DataFrames in dict_list to form a final DataFrame if needed
dist_df = pd.concat(dict_list, ignore_index=True)


# Prepare random forest classifier

## Random forest specific functions

In [ ]:
def normal_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return np.random.normal(variable_mean, variable_sd, n)

def uniform_generator(n, min_val, max_val, min_variation, max_variation):
    variable_min = min_val + np.random.uniform(-min_variation * 2, min_variation * 2)
    variable_max = max_val + np.random.uniform(-max_variation * 2, max_variation * 2)
    return np.random.uniform(variable_min, variable_max, n)

def right_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return np.random.lognormal(variable_mean, variable_sd, n)

def left_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return -np.random.lognormal(variable_mean, variable_sd, n)

def multimodal_generator(ns, means, sds, mean_variation, sd_variation):
    samples_list = []
    for i in range(len(means)):
        variable_mean = means[i] + np.random.normal(0, mean_variation * 2)
        variable_sd = sds[i] + abs(np.random.normal(0, sd_variation * 2))
        samples_list.append(np.random.normal(variable_mean, variable_sd, ns[i]))
    return np.concatenate(samples_list)

In [ ]:
def rescale_to_minus_one_one(values):
    min_value = np.min(values)
    max_value = np.max(values)
    scaled_values = 2 * ((values - min_value) / (max_value - min_value)) - 1
    return scaled_values

In [ ]:
def generate_samples(n, q, generator_func, *args):
    samples = generator_func(n, *args)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)

def generate_normal_samples(n=10000, q=10, mean_value=0, sd_value=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, normal_generator, mean_value, sd_value, mean_variation, sd_variation)

def generate_uniform_samples(n=10000, q=10, min_value=-1, max_value=1, min_variation=0.5, max_variation=0.5):
    return generate_samples(n, q, uniform_generator, min_value, max_value, min_variation, max_variation)

def generate_right_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, right_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_left_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, left_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_multimodal_samples(ns=[1000, 50], q=10, means=[1, -1], sds=[1, 0.5], mean_variation=0.5, sd_variation=0.5):
    samples = multimodal_generator(ns, means, sds, mean_variation, sd_variation)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)


In [ ]:
def generate_training_data(n_samples=10000):
    data_list = {}

    # Generate samples for each distribution type
    data_list['normal'] = [generate_normal_samples() for _ in range(n_samples)]
    data_list['bimodal'] = [generate_multimodal_samples() for _ in range(n_samples)]
    data_list['uniform'] = [generate_uniform_samples() for _ in range(n_samples)]
    data_list['right_tailed'] = [generate_right_tailed_samples() for _ in range(n_samples)]
    data_list['left_tailed'] = [generate_left_tailed_samples() for _ in range(n_samples)]

    # Combine all data into a single DataFrame
    combined_data = pd.DataFrame()
    for name, samples_list in data_list.items():
        df = pd.DataFrame(samples_list)
        df['label'] = name
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    # Assign column names
    combined_data.columns = [f'V{i+1}' for i in range(combined_data.shape[1] - 1)] + ['label']

    return combined_data

## Train random forest classifier

In [ ]:
training_data = generate_training_data()

In [101]:
dist_df['imp_c_quantile_scaled'] = [rescale_to_minus_one_one(dist) for dist in dist_df["imp_c_quantile"]]

In [103]:
dist_df.columns

Index(['dist_id', 'labeled_dist_h', 'labeled_dist_a', 'imp_c',
       'imp_c_quantile', 'imp_c_quantile_scaled'],
      dtype='object')

In [158]:
y_true = label_encoder.inverse_transform(y_encoded).tolist()

In [160]:
# y_true

In [163]:
# [print(i) for i in zip(y_true, y_pred_list)]

In [171]:
# Separate features and labels
X = pd.DataFrame(dist_df['imp_c_quantile_scaled'].tolist(), columns=[f'col{i}' for i in range(1, 11)])
y = dist_df['labeled_dist_h']
# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_pred_list = []
for i in range(len(X)):


  # Initialize the Random Forest Classifier
  rf_model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, min_samples_leaf=2, random_state=123)

  # Train the model
  rf_model.fit(np.array(X.drop(index= i)), np.concatenate([y[:i], y[i+1:]]))

  # Print the trained model
  print(rf_model)

  # Predict the responses for test dataset
  y_pred =  rf_model.predict(X)[i]
  y_pred_list.append(y_pred)

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [180]:
# training_data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,-1.0,-0.332937,-0.200880,-0.101343,-0.012327,0.064239,0.147099,0.237064,0.361285,1.0
1,-1.0,-0.180681,-0.059398,0.024828,0.103016,0.177555,0.252776,0.335356,0.454815,1.0
2,-1.0,-0.306321,-0.185252,-0.100673,-0.022990,0.048403,0.122758,0.211408,0.329563,1.0
3,-1.0,-0.271461,-0.151694,-0.063337,0.011303,0.088440,0.166409,0.258503,0.382444,1.0
4,-1.0,-0.237897,-0.111964,-0.026225,0.052634,0.124292,0.198891,0.287789,0.406175,1.0
...,...,...,...,...,...,...,...,...,...,...
49995,-1.0,0.978290,0.990525,0.994895,0.997011,0.998241,0.998941,0.999424,0.999751,1.0
49996,-1.0,0.926075,0.960536,0.974478,0.983221,0.988601,0.992467,0.995269,0.997473,1.0
49997,-1.0,0.986333,0.994304,0.997018,0.998354,0.999055,0.999481,0.999726,0.999892,1.0
49998,-1.0,0.987097,0.994767,0.997125,0.998306,0.998962,0.999404,0.999667,0.999853,1.0


In [195]:
y_pred_list= rf_model.predict(training_data.drop('label', axis=1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [202]:
y_true = [label.replace("_", " ") for label in training_data['label']]

In [203]:
y_true

['normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',

In [204]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.model_selection import cross_val_score
import numpy as np

# Assuming you have the following:
# X_test - your test features
# y_test - your true test labels
# model - your trained classifier

# Predict the responses for test dataset
# y_pred =  label_encoder.inverse_transform(rf_model.predict(X))
# y_true, y_pred_list
# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_list))

# Accuracy
print("Accuracy:", accuracy_score(y_true, y_pred_list))

# Precision, Recall, F1-Score
print("Classification Report:")
print(classification_report(y_true, y_pred_list))

# If it's a binary classification, calculate ROC-AUC
if len(np.unique(y_pred_list)) == 2:
    y_probs = rf_model.predict_proba(X_test)[:, 1]  # probability estimates for the positive class
    fpr, tpr, thresholds = roc_curve(y_pred_list, y_probs)
    print("AUC:", auc(fpr, tpr))

# Cross-validation (if needed)
# cv_scores = cross_val_score(model, X, y, cv=5)  # X, y are the full dataset
# print("Cross-validation scores:", cv_scores)
# print("Mean cross-validation score:", np.mean(cv_scores))


Confusion Matrix:
[[  531  1389  8073     7     0]
 [    0 10000     0     0     0]
 [   19     4  9977     0     0]
 [    0     0     0 10000     0]
 [    0     0     0     0 10000]]
Accuracy: 0.81016
Classification Report:
              precision    recall  f1-score   support

     bimodal       0.97      0.05      0.10     10000
 left tailed       0.88      1.00      0.93     10000
      normal       0.55      1.00      0.71     10000
right tailed       1.00      1.00      1.00     10000
     uniform       1.00      1.00      1.00     10000

    accuracy                           0.81     50000
   macro avg       0.88      0.81      0.75     50000
weighted avg       0.88      0.81      0.75     50000



# Cross validation loop

In [122]:
rf_model

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)

In [ ]:
# scaled_numeric_df

NameError: name 'scaled_numeric_df' is not defined

In [24]:
# Cross-validation: loop through all dist_ids, one at a time
# subset data to exclude unlabeled distributions 1st

data = numeric_df.dropna(subset=['dist_id'])

unique_dist_ids = data['dist_id'].unique()
np.random.shuffle(unique_dist_ids)


result_list = []

# Loop over each dist_id, leaving one out at time
for xval_id in unique_dist_ids:
    r2_list = []
    loss_list = []
    prediction_list = []
    matched_data_list = []
    model_list = []
    # Reserve data for validation
    xval_data = data[data['dist_id'] == xval_id]

    # Matched data for training
    train_data = data[data['dist_id'] != xval_id]

    reserve_data = train_data.groupby('unique_id').sample(n=1)

    for i in range(10):
        matched_data = train_data.groupby('unique_id').sample(n=1)
        matched_data = matched_data.reset_index(drop=True)

        model, test_loss, r_squared, predictions, y_test = train_model_a(matched_data)

        predictions = predict_on_reserve(reserve_data, model, min_c, max_c)

        matched_data_list.append(matched_data)
        prediction_list.append(predictions)
        model_list.append(model)
        loss_list.append(test_loss)
        r2_list.append(r_squared)

    combined_array = np.array([predictions['predictions'].values for predictions in prediction_list])
    combined_array[combined_array < 0 ] = 0

    arr_min = np.min(combined_array.flatten())
    arr_max = np.max(combined_array.flatten())

    dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
    norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    scaled_numeric_df = prediction_list[0][keep_cols]

    scaled_numeric_df['norm_dist_array'] = norm_dist_array
    scaled_numeric_df.dropna(inplace=True)

    scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()

    norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

    X = scaled_numeric_df[keep_cols]
    y = norm_dist_array

    # X = scaled_numeric_df.iloc[:, :-1]
    # y = scaled_numeric_df.iloc[:, -1]

    # x = np.array(X)
    # y = np.array(norm_dist_array)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = build_model(X_train.shape[1],output_shape=10)

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
    model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
    model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)


    predictions = model.predict(xval_data[keep_cols])
    imp_c_scaled = np.sort(xval_data.groupby('unique_id').sample(n=1)['imp_c_scaled']*max_c+min_c)

    imp_c_quantiles = np.quantile(imp_c_scaled, np.linspace(0, 1,10))

    scaled_predictions = predictions*max_c+min_c
    prediction_quantiles = [np.quantile(prediction, np.linspace(0, 1,10)) for prediction in scaled_predictions]

    prediction_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  predictions]

    # Use the trained model to make predictions
    predicted_labels_encoded = rf_model.predict(prediction_rf_ready_predictions)

    # Decode the encoded labels back to original labels
    predicted_predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded)

    real_rf_ready = [rescale_to_minus_one_one(sorted(prediction)+ np.random.uniform(-0.001, 0.001, prediction.shape[0])) for prediction in  [imp_c_quantiles]]

    # Use the trained model to make predictions
    real_labels_encoded = rf_model.predict(real_rf_ready)


    # Decode the encoded labels back to original labels
    real_predicted_labels = label_encoder.inverse_transform(real_labels_encoded)

    lat_lon = [Point(xy) for xy in zip(xval_data['latitude'],xval_data['longitude'])]
    xy_sample = [Point(xy) for xy in zip(xval_data['x_sample'],xval_data['y_sample'])]
    xy_driver = [Point(xy) for xy in zip(xval_data['x_driver'],xval_data['y_driver'])]

    result_dict ={
                  "xval_id": xval_id,
                  "lat_lon": lat_lon,
                  "xy_sample": xy_sample,
                  "xy_driver": xy_driver,
                  "imp_c_scaled": imp_c_scaled,
                  "imp_c_quantiles": imp_c_quantiles,
                  "real_predicted_labels": real_predicted_labels,
                  "scaled_predictions": scaled_predictions,
                  "prediction_quantiles": prediction_quantiles,
                  "predicted_predicted_labels": predicted_predicted_labels
                }
    result_list.append(result_dict)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 933ms/step - loss: 2.8913 - val_loss: 0.0063
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 2.4649 - val_loss: 0.0058
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.3473 - val_loss: 0.0075
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.7608 - val_loss: 0.0025
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.4367 - val_loss: 0.0107
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.6386 - val_loss: 0.0055
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.4006 - val_loss: 0.0049
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.4626 - val_loss: 0.0069
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.3778 - val_loss: 0.0047
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.5440 - val_loss: 0.0046
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.8696 - val_loss: 0.0064
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0290 - val_l

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 2.3823 - val_loss: 0.0146
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.1491 - val_loss: 0.0230
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.0117 - val_loss: 0.0041
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.8738 - val_loss: 0.0062
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.6781 - val_loss: 0.0059
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.4216 - val_loss: 0.0049
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.2513 - val_loss: 0.0069
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.4530 - val_loss: 0.0040
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0074 - val_loss: 0.0039
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9785 - val_loss: 0.0052
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9699 - val_loss: 0.0044
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9232 - val_loss: 0.0049
Epoch 13/10

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 3.9835 - val_loss: 0.0083
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.6016 - val_loss: 0.0074
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.3465 - val_loss: 0.0032
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.5247 - val_loss: 0.0050
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.6004 - val_loss: 0.0036
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.5945 - val_loss: 0.0037
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.5708 - val_loss: 0.0060
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.1573 - val_loss: 0.0057
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.2573 - val_loss: 0.0053
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.1096 - val_loss: 0.0055
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.1572 - val_loss: 0.0056
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.1431 - val_loss: 0.0057
Epo

<ipython-input-24-c852c45d897e>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df['norm_dist_array'] = norm_dist_array
<ipython-input-24-c852c45d897e>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df.dropna(inplace=True)
<ipython-input-24-c852c45d897e>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 2.2000 - mae: 1.1195 - val_loss: 0.0168 - val_mae: 0.0905
Epoch 2/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 1.9030 - mae: 1.0452 - val_loss: 0.0142 - val_mae: 0.0802
Epoch 3/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.7482 - mae: 1.0136 - val_loss: 0.0129 - val_mae: 0.0740
Epoch 4/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 1.6998 - mae: 0.9985 - val_loss: 0.0123 - val_mae: 0.0705
Epoch 5/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.7317 - mae: 1.0157 - val_loss: 0.0121 - val_mae: 0.0690
Epoch 6/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1.6587 - mae: 1.0027 - val_loss: 0.0121 - val_mae: 0.0686
Epoch 7/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.5784 - mae: 0.9554 - val_loss: 0.0120 - val_mae: 0.0688
Epoch 8/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.6215 - mae: 0.9734 - val_loss: 0.0121 - val_mae: 0.0697
Epoch 9/600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.5974 - mae: 0.9569 - val_l

KeyboardInterrupt: 

In [ ]:
result_list[4]['imp_c_quantiles']


In [ ]:
gdf = gpd.GeoDataFrame({'median ': [i[5]for i in result_list[4]['prediction_quantiles']],'median_z': [i[5]-result_list[4]['imp_c_quantiles'][5] for i in result_list[4]['prediction_quantiles']],'geometry': result_list[4]['xy_driver']})
gdf.set_crs(epsg = 32604, inplace=True)
gdf.to_file('test.gpkg')

In [ ]:
# Calculate average R-squared and loss
average_r_squared = np.mean(r2_list)
average_loss = np.mean(loss_list)

print(average_r_squared)
print(average_loss)

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(r2_list, bins=20, color='blue', alpha=0.7)
plt.title('R-squared Distribution')
plt.xlabel('R-squared')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(loss_list, bins=20, color='red', alpha=0.7)
plt.title('Loss Distribution')
plt.xlabel('Loss')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
dist_id_performance = {}
for i, dist_id in enumerate(unique_dist_ids):
    if dist_id not in dist_id_performance:
        dist_id_performance[dist_id] = []
    dist_id_performance[dist_id].append(r2_list[i])

# Print R-squared by dist_id
for dist_id, performances in dist_id_performance.items():
    print(f"Dist_id {dist_id} - Average R-squared: {np.mean(performances)}")